## Set Up

### Import dependencies

Run the below cell to import Python dependencies needed for displaying the results in this notebook
(tip: select the cell and use **Ctrl+enter** to run the cell)

In [1]:
from IPython.display import HTML
import matplotlib.pyplot as plt
import os
import time
import sys                                     
from pathlib import Path
sys.path.insert(0, str(Path().resolve().parent.parent))
from demoTools.demoutils import *

## Using Intel® Distribution of OpenVINO™ toolkit

First, let's try running inference on a single image to see how the Intel® Distribution of OpenVINO™ toolkit works.
We will be using Intel® Distribution of OpenVINO™ toolkit Inference Engine (IE) to locate vehicles on the road.
There are five steps involved in this task:

1. Create an Intermediate Representation (IR) Model using the Model Optimizer by Intel
2. Choose a device and create IEPlugin for the device
3. Read the IRModel using IENetwork
4. Load the IENetwork into the Plugin
5. Run inference.

### Creating IR Model

The Model Optimizer creates Intermediate Representation (IR) models that are optimized for different end-point target devices.
These models can be created from existing DNN models from popular frameworks (e.g. Caffe*, TF) using the Model Optimizer. 

The Intel® Distribution of OpenVINO™ toolkit includes a utility script `model_downloader.py` that you can use to download some common modes. Run the following cell to see the models available through `model_downloader.py`

In [2]:
!/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --print_all

densenet-121
densenet-161
densenet-169
densenet-201
squeezenet1.0
squeezenet1.1
mtcnn-p
mtcnn-r
mtcnn-o
mobilenet-ssd
vgg19
vgg16
ssd512
ssd300
inception-resnet-v2
dilation
googlenet-v1
googlenet-v2
googlenet-v4
alexnet
ssd_mobilenet_v2_coco
resnet-50
resnet-101
resnet-152
googlenet-v3
se-inception
se-resnet-101
se-resnet-152
se-resnet-50
se-resnext-50
se-resnext-101
Sphereface
license-plate-recognition-barrier-0007
mobilenet-v1-1.0-224
mobilenet-v2
faster_rcnn_inception_v2_coco
deeplabv3
ctpn
ssd_mobilenet_v1_coco
faster_rcnn_resnet101_coco
mobilenet-v2-1.4-224
age-gender-recognition-retail-0013
age-gender-recognition-retail-0013-fp16
emotions-recognition-retail-0003
emotions-recognition-retail-0003-fp16
face-detection-adas-0001
face-detection-adas-0001-fp16
face-detection-retail-0004
face-detection-retail-0004-fp16
face-person-detection-retail-0002
face-person-detection-retail-0002-fp16
face-reidentification-retail-0095
face-reidentification-retail-0095-fp16
facial-landmarks-35-adas-

**Note** the '!' is a special Jupyter Notebook command that allows you to run shell commands as if you are in a commannd line. So the above command will work straight out of the box on in a terminal (with '!' removed).

In this demo, we will be using the **dilation** model. This model can be downloaded with the following command.

In [3]:
!/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --name dilation -o raw_models


###############|| Downloading topologies ||###############

========= Downloading raw_models/semantic_segmentation/dilation/cityscapes/caffe/dilation.prototxt
... 100%, 9 KB, 36300 KB/s, 0 seconds passed

========= Downloading raw_models/semantic_segmentation/dilation/cityscapes/caffe/dilation.caffemodel
... 100%, 524898 KB, 28131 KB/s, 18 seconds passed


###############|| Post processing ||###############



The input arguments are as follows:
* --name : name of the model you want to download. It should be one of the models listed in the previous cell
* -o : output directory. If this directory does not exist, it will be created for you.

There are more arguments to this script and you can get the full list using the `-h` option.


With the `-o` option set as above, this command downloads the model in the directory `raw_models`, with the `.caffemodel` located at `raw_models/semantic_segmentation/dilation/cityscapes/caffe/dilation.caffemodel`

Now, let's convert this to the optimized model using the model optimizer.

### Creating job file

All the code up to this point has been run within the Jupyter Notebook instance running on a development node based on an Intel® Xeon® Scalable processor, where the Notebook is allocated a single core. 
To run inference and convert model we need more compute power and disk space.
We will run the workload on edge compute nodes represented in the IoT DevCloud. We will send work to the edge compute nodes by submitting the corresponding non-interactive jobs into a queue. For each job, we will specify the type of the edge compute server that must be allocated for the job.

The job file is written in Bash, and will be executed directly on the edge compute node.
For this example, we have written the job files for you in the notebook.

In [4]:
%%writefile convert_model.sh

cd $PBS_O_WORKDIR
python3 /opt/intel/openvino/deployment_tools/model_optimizer/mo.py \
--framework caffe \
--input_shape [1,3,1396,1396] \
--output prob \
--input_model raw_models/semantic_segmentation/dilation/cityscapes/caffe/dilation.caffemodel \
--input_proto raw_models/semantic_segmentation/dilation/cityscapes/caffe/dilation.prototxt \
--data_type FP32 \
-o models/dilation/FP32 \
--mean_values [72.39,82.91,73.16] 

Writing convert_model.sh


### Understand how jobs are submitted into the queue

Now that we have the job script, we can submit the jobs to edge compute nodes. In the IoT DevCloud, you can do this using the `qsub` command.
We can submit object_detection_job to several different types of edge compute nodes simultaneously or just one node at a time.

There are three options of `qsub` command that we use for this:
- `-l` : this option lets us select the number and the type of nodes using `nodes={node_count}:{property}`. 
- `-F` : this option lets us send arguments to the bash script. 
- `-N` : this option lets use name the job so that it is easier to distinguish between them.

#### Submitting to an edge compute node with an Intel® CPU
In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank* 870-Q170</a> edge node with an <a 
    href="https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz-">Intel® Core™ i5-6500TE processor</a>.
    The output of cell below is job id that is used for checking status of the job.

In [5]:
job_id = !qsub convert_model.sh -l nodes=1:tank-870:i5-6500te
print(job_id)

['34352.c003']


### Check if the jobs are done

To check on the jobs that were submitted, use the `qstat` command.

We have created a custom Jupyter widget  to get live qstat update.
Run the following cell to bring it up. 

In [6]:
liveQstat()

Output(layout=Layout(border='1px solid gray', width='100%'))

Button(description='Stop', style=ButtonStyle())

You should see the jobs you have submitted (referenced by `Job ID` that gets displayed right after you submit the job.
There should also be an extra job in the queue "jupyterhub": this job runs your current Jupyter Notebook session.

The 'S' column shows the current status. 
- If it is in Q state, it is in the queue waiting for available resources. 
- If it is in R state, it is running. 
- If the job is no longer listed, it means it is completed.

**Note**: Time spent in the queue depends on the number of users accessing the edge nodes. Once these jobs begin to run, they should take from 1 to 5 minutes to complete. 

## Submit Inference

After model convertion is finished we can submit job for inference model. Following cells create job script for inference, submit job to the server, displays statuses of current jobs and the final one displays original image and result of inference. You can use shift+enter hot key in order to start cell one by one faster. Please, note, that inference should be finished before running last cell.

In [7]:
%%writefile semantic_segmentation.sh

cd $PBS_O_WORKDIR
python3 aist2019-openvino-semseg/semseg_inference.py \
--model=models/dilation/FP32/dilation.xml            \
--weights=models/dilation/FP32/dilation.bin          \
--input=data                                         \
--batch_size=1                                       \
--device="CPU"                                       \
--color_map=aist2019-openvino-semseg/color_map.txt

Writing semantic_segmentation.sh


In [8]:
job_id_core = !qsub semantic_segmentation.sh -l nodes=1:tank-870:i5-6500te -N sem_segm
print(job_id_core[0])

34354.c003


In [9]:
liveQstat()

Output(layout=Layout(border='1px solid gray', width='100%'))

Button(description='Stop', style=ButtonStyle())

In [ ]:
import cv2

def show_img(name):
    fig = plt.figure(dpi=300)
    ax = fig.add_subplot(111)
    ax.imshow(cv2.cvtColor(cv2.imread(name), cv2.COLOR_BGR2RGB), interpolation='none')
    
    plt.axis("off")
    plt.show()
    
show_img('data/test.jpg')
show_img('aist2019-openvino-semseg/out_segmentation_0.bmp')